In [6]:
import numpy as np


# to read the training data and making use of vocabulary and index the chars:
# To read the training data and make a vocabulary and dictiornary to index the chars
class DataReader:
    def __init__(self, path, seq_length):
        #uncomment below , if you dont want to use any file for text reading and comment next 2 lines
        #self.data = "some really long text to test this. maybe not perfect but should get you going."
        self.fp = open(path, "r")
        self.data = self.fp.read()
        #find unique chars
        chars = list(set(self.data))
        #create dictionary mapping for each char
        self.char_to_ix = {ch:i for (i,ch) in enumerate(chars)}
        self.ix_to_char = {i:ch for (i,ch) in enumerate(chars)}
        #total data
        self.data_size = len(self.data)
        #num of unique chars
        self.vocab_size = len(chars)
        self.pointer = 0
        self.seq_length = seq_length

    def next_batch(self):
        input_start = self.pointer
        input_end = self.pointer + self.seq_length
        inputs = [self.char_to_ix[ch] for ch in self.data[input_start:input_end]]
        targets = [self.char_to_ix[ch] for ch in self.data[input_start+1:input_end+1]]
        self.pointer += self.seq_length
        if self.pointer + self.seq_length + 1 >= self.data_size:
            # reset pointer
            self.pointer = 0
        return inputs, targets

    def just_started(self):
        return self.pointer == 0

    def close(self):
        self.fp.close()
        


In [37]:
class RNN:
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        # model parameters
        self.U = np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hidden_size, vocab_size))
        self.V = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))
        self.W = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))
        self.b = np.zeros((hidden_size, 1)) # bias for hidden layer
        self.c = np.zeros((vocab_size, 1)) # bias for output
        
        # memory vars for adagrad, 
        #ignore if you implement another approach
        self.mU = np.zeros_like(self.U)
        self.mW = np.zeros_like(self.W)
        self.mV = np.zeros_like(self.V)
        self.mb = np.zeros_like(self.b)
        self.mc = np.zeros_like(self.c)

    def softmax(self, x):
        p = np.exp(x- np.max(x))
        return p / np.sum(p)
        
    def forward(self, inputs, hprev):
        xs, hs, os, ycap = {}, {}, {}, {}
        hs[-1] = np.copy(hprev)
        for t in range(len(inputs)):
            xs[t] = np.zeros((self.vocab_size,1))
            xs[t][inputs[t]] = 1 # one hot encoding , 1-of-k
            hs[t] = np.tanh(np.dot(self.U,xs[t]) + np.dot(self.W,hs[t-1]) + self.b) # hidden state
            os[t] = np.dot(self.V,hs[t]) + self.c # unnormalised log probs for next char
            ycap[t] = self.softmax(os[t]) # probs for next char
        return xs, hs, ycap
        
    def backward(self, xs, hs, ps, targets):
        # backward pass: compute gradients going backwards
        dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
        db, dc = np.zeros_like(self.b), np.zeros_like(self.c)
        dhnext = np.zeros_like(hs[0])
        for t in reversed(range(self.seq_length)):
            dy = np.copy(ps[t])
            #through softmax
            dy[targets[t]] -= 1 # backprop into y
            #calculate dV, dc
            dV += np.dot(dy, hs[t].T)
            dc += dc
            #dh includes gradient from two sides, next cell and current output
            dh = np.dot(self.V.T, dy) + dhnext # backprop into h
            # backprop through tanh non-linearity 
            dhrec = (1 - hs[t] * hs[t]) * dh  #dhrec is the term used in many equations
            db += dhrec
            #calculate dU and dW
            dU += np.dot(dhrec, xs[t].T)
            dW += np.dot(dhrec, hs[t-1].T)
            #pass the gradient from next cell to the next iteration.
            dhnext = np.dot(self.W.T, dhrec)
        # clip to mitigate exploding gradients
        for dparam in [dU, dW, dV, db, dc]:
            np.clip(dparam, -5, 5, out=dparam) 
        return dU, dW, dV, db, dc
    
    def loss(self, ps, targets):
        """loss for a sequence"""
        # calculate cross-entrpy loss
        return sum(-np.log(ps[t][targets[t],0]) for t in range(self.seq_length))
        
    
    def update_model(self, dU, dW, dV, db, dc):
        # parameter update with adagrad
        for param, dparam, mem in zip([self.U, self.W, self.V, self.b, self.c],
                                  [dU, dW, dV, db, dc],
                                  [self.mU, self.mW, self.mV, self.mb, self.mc]):
            mem += dparam*dparam
            param += -self.learning_rate*dparam/np.sqrt(mem+1e-8) # adagrad update
                
                
    def sample(self, h, seed_ix, n):
        """
        sample a sequence of integers from the model
        h is memory state, seed_ix is seed letter from the first time step
        """
        x = np.zeros((self.vocab_size, 1))
        x[seed_ix] = 1
        ixes = []
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = np.zeros((self.vocab_size,1))
            x[ix] = 1
            ixes.append(ix)
        return ixes

    def train(self, data_reader):
        iter_num = 0
        threshold = 0.00168
        smooth_loss = -np.log(1.0/data_reader.vocab_size)*self.seq_length
        while (smooth_loss > threshold):
            if data_reader.just_started():
                hprev = np.zeros((self.hidden_size,1))
            inputs, targets = data_reader.next_batch()
            xs, hs, ps = self.forward(inputs, hprev)
            dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
            loss = self.loss(ps, targets)
            self.update_model(dU, dW, dV, db, dc)
            smooth_loss = smooth_loss*0.999 + loss*0.001
            hprev = hs[self.seq_length-1]
            if not iter_num%500:
                sample_ix = self.sample(hprev, inputs[0], 200)
                print( ''.join(data_reader.ix_to_char[ix] for ix in sample_ix))
                print( "\n\niter :%d, loss:%f"%(iter_num, smooth_loss))
            iter_num += 1

    def predict(self, data_reader, start, n):
        

        #initialize input vector
        x = np.zeros((self.vocab_size, 1))
        chars = [ch for ch in start]
        ixes = []
        for i in range(len(chars)):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            ixes.append(ix)

        h = np.zeros((self.hidden_size,1))
        # predict next n chars
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = np.zeros((self.vocab_size,1))
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt

In [38]:
seq_length = 25
#read text from the "input.txt" file
data_reader = DataReader("temp.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
rnn.train(data_reader)

ilfbhe tacztront
cgv  y ftaerlgr oathsanhouuhog aoeyit 
rl difatao on
aug eol
bozlyfeonl caebo catwlvcztgovh two
yag ta
bhz hwzofiltzhrn yhoftfont sntetef
ofg l l
tatg
itgriye r selct la atdlymatimsgi


iter :0, loss:78.387613
inguandsc
ngn thesfa zove masecabtses s lnonftg
tuiufts zn lhgsdole sesthe sesvfanisg
s ine ineure gas s are dn teesle in inltwe cag cei fole l
lgoinggle tuvesg
t
shnese ttldnl in the uain  bn camhear


iter :500, loss:68.650863
  leas on the m loce seareod can cas cas le the am the machine ire cacme  inithe azi l
ls in cage thi ling one mat
i love matels ses are westhi e s
s inftwe uattst
l chtts in the cas s ldast
sanehi lo


iter :1000, loss:49.316733
 leatuscazing
one sho cob ure c
st
 ab
thine matt dses cattulonis love codingulicrisearn litri we the fats ino
ine l ng
e in t
i vels fre awesome
technical use cases in cle s s are fat
i love chate tr


iter :1500, loss:32.930831
inb awesomfrninhingtli socann ing
onineninathf ingncon chiching
lan cait
ining
in ch

ets in the east
language models are awesome
technical use cases are amazing
i love coding
one should leate  mat inld love ses are awes in the east
language models are awesome
technical use cases are a


iter :18000, loss:0.085283
langulg srasesome
technical use cases are amazing
i love coding
one should leat
s moge shin cat  ing
i lode shouases in the east
language models are awesome
technical use cases are amazing
i love codi


iter :18500, loss:0.082037
ing de sr
sun the doved thful
can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should leats can climb trees
sun 


iter :19000, loss:0.079015
use cangulgeliclire seliin se can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amezing
one should in the east
language models are awesome



iter :19500, loss:0.076249
tulihn mat
i love studying machine learning
dogs are faithful
cats can climb tre

s s ld love souzyine
dogs are f
i love coding
one should learning
dogs are faithful
cats can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are 


iter :36000, loss:0.349965
lang
ine should learning
don chise
sus t are awesome
technical use cases are amazing
i love coding
one should leate che east
language models are awesome
technical use cases are amazing
i love coding
o


iter :36500, loss:0.244527
ing
dong
i love coding
one should learning
dogs are faithful
cats can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
o


iter :37000, loss:0.177742
use cases are amazing
i love coding
one should learning
dogs are faithful
cats can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i 


iter :37500, loss:0.135075
tst
fatt she awesome
technical use cases are amazing
i love coding
one should le

gts ioseovels are awesome
technical use cases are amazing
i love codinailove coding
one should learning
dogs are faithful
cats can climb trees
sun sets in the west
sun rises in the east
language model


iter :54000, loss:0.029616
lang mats can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should leats can climb trees
sun sets in the west
sun


iter :54500, loss:0.029164
ing
don sets in reesdinacess chn castulanguage models are awesome
technical use cases are amazing
i love coding
one should leats can climb trees
sun sets in the west
sun rises in the east
language mod


iter :55000, loss:0.028722
use cangulg srin the east
language models are awesome
technical use cases are amazing
i love coding
one should leare sai onelonguame asts in the west
sun rises in the east
language models are awesome



iter :55500, loss:0.028308
tse faithful
cats can climb trees
sun sets in the west
sun rises in the east
lan

ove studying machine learning
dogs are faithful
cats can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should lea


iter :72000, loss:0.019453
lfats ine should leats can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should leat
i love studying machine lear


iter :72500, loss:0.019281
n techning
donise
sa
oul csesome
technical use cases are amazing
i love coding
one should leare sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one shou


iter :73000, loss:0.019105
use cang laithful
cat fonguage mode swomaine learning
dogs are faithful
cats can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i lo


iter :73500, loss:0.018943

s ses are amazing
i love coding
one should leats can climb trees
sun setstr so


...

ove studying machine learning
dogs are west
sun rises in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should leate omen the west
sun ris


iter :558000, loss:0.001925
language chets in the west
sun rises in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should leate che east
language models on the wels s


iter :558500, loss:0.001924
ing
i love coding
one should leases are amazing ihoul cading
one should leate che east
language models on theude scare faithful
cats can climb trees
sun sets in the west
sun rises in the east
language


iter :559000, loss:0.001921
usets ioul cldinearning
doge  wome
technical use cases are amazing
i love coding
one should leate cate che doolease ome t
sun none should leate che dovedonimb  wose
sareas
s le thful
cats can cage sov


iter :559500, loss:0.001920
e s arning
dogs arees
sun sets in the east
language models are awesome
techn

oveosts are amazing
i love coding
one should leate che east
language models as in trees
sun sets in the dooscane deose
cate che east
language models are awes are amazing
i love coding
one should leate


iter :576000, loss:0.001859
language models are uwesome
technical use cases are amazing
i love coding
one should leate che east
language models on the mats can climb trees
sun sets in the west
sun rises in the west
sun rises in 


iter :576500, loss:0.001857
ing
i love chs
s ioul cat
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should leate ome cwess doge cwesetre amazing
i love c


iter :577000, loss:0.001855
 s ts in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should leate che east
language models on the mats can climb trees
sun sets in the 


iter :577500, loss:0.001854
e fun uate omen the west
sun rises in the east
language models are awesome
t

ove studying machine lase che faithful
cats can climb trees
sun sets in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one should leats can cl


iter :594000, loss:0.001797
language models are awest
dels
onethe mat
i love studying machine lang ine shong
donechn tn the awesome tesss are amazing
i love coding
one should leate che east
language models on the wels soue ses a


iter :594500, loss:0.001796
dogs are fni on the machine learning
dogs are faithful
cat ooltulove s udniset areises arees
canove models on the east
language models are awesome
technical use cases are amazing
i love coding
one sho


iter :595000, loss:0.001793
use case che uoding
i eaimat
i love studying machine learning
dogs are west
sun rises in the west
sun rises in the east
language models are awesome
technical use cases are amazing
i love coding
one sh


iter :595500, loss:0.001793
n  in chithing
ine in the fast
language models are awesome
technical use cas

ove studying machine learning
dogs are faithful
cat eore fuatuone west
sun uselong
done should leate che eastulanguage models are awesome
technical use cases are amazing
i love coding
one should leate


iter :612000, loss:0.001739
language models are awest
d cate chets uayisose
catoule lesteehleases are amazing
i love coding
one should leats can climb trees
sun sets in the west
sun rises in the east
language models are awesome



iter :612500, loss:0.001738
n te elodels are awes in the uas
i learning
doet trding
onethise
ca
s learning
dogs are west
sun rises are fal
uad
sun rises seare one i ldvl studose shouases are amazing
i lovi eose
san ul tr inguage


iter :613000, loss:0.001736
usets in the east
language models are awesome
technical use cases are amazing
i love coding
one should leate che east
language models on the mats can climb trees
sun sets in the west
sun rises in the 


iter :613500, loss:0.001735
tst
done should leate che east
language models are awesome
technical use cas

ove studying machine lsonguazingulove coding
one should leate che ould leate cale so
eases are amazing
i love coding
one should leate che east
language models are awesome
technical use cases are amazi


iter :630000, loss:0.001685
ingul
ing
ine should leate che onets oae che sou nathin  rodels are awesome
technical use cases are amazing
i love coding
one should leatt language coding
one sel mb ts in the daste iining mat
i love 


iter :630500, loss:0.001683
n thchnical use cases are amazing
i love coding
one should leat
i ldve studying malodels are awes in the east
language models are awesome
technical use cases are amazing
i love coding
one should leat



iter :631000, loss:0.001681
use cangulaithful
cats chn the east
language models are awesome
technical use cases are amazing
i love coding
one should leate che east
language models on the wes usai es inamazing
i lose chetu renimb


iter :631500, loss:0.001681


In [36]:
rnn.predict(data_reader, 'sun rises', 3)

'sun risesls '